In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [4]:
import pathlib

In [3]:
def get_images_directory(directory):
    data_dir = pathlib.Path(directory)
    return data_dir

In [11]:
data_dir=get_images_directory('../raw_data/fungi_identification/images')

In [14]:
data_dir

PosixPath('../raw_data/fungi_identification/images')